In [16]:
import os
import json
import pandas
import numpy as np
from dateutil import parser
from argparse import ArgumentParser

In [2]:
threshold = 0.9
after = '0610'

In [3]:
def filter_log_time(log):
    success = False
    times = [parser.parse(log[i-1].split(',')[0]) for i in range(len(log)) if 'Best Action' in log[i]]
    return times
def get_intervals(times):
    delta_time = []
    for i in range(len(times)-1):
        delta = times[i+1] - times[i]
        delta_time.append(delta)
    return delta_time
def filter_log(log):
    success = False
    log = [float(t.replace('\n', '').split(' ')[-1]) for t in log if 'Score' in t]
    if len(log)<2:
        return None
    if log[-1]==log[-2]:
        #if log[-1]>args.threshold:
        #    success = True
        log.pop(-1)
    best_score = np.max(log)
    if np.max(log)>threshold:
        success = True
        end = log.index(best_score)+1
        log = log[:end]
    log.append(success)
    return log

In [6]:
logs = sorted([d for d in os.listdir('data') if d.startswith('mcts') or d.startswith('Sub')])
logs = [d for d in logs if '.zip' not in d]
for logname in logs:
    print(os.path.join('data', logname))

data/Sub-circle-1.0-0715_2147
data/Sub-line-1.0-0715_2033
data/mcts-0306_2343
data/mcts-0306_2346
data/mcts-0309_1757
data/mcts-0310_1132
data/mcts-0520_2359
data/mcts-0715_2101
data/mcts-0715_2112
data/mcts-0715_2121
data/mcts-0715_2129
data/mcts-0715_2132
data/mcts-0715_2348
data/mcts-0716_0028
data/mcts-0724_0117
data/mcts-0724_0501


In [11]:
with open(os.path.join('data', logname, 'config.json'), 'r') as f:
    cfg = json.load(f)

In [14]:
cfg['iql_path']

'/disk1/hogun/PreferenceDiffusion/iql/logs/0420_1905/iql_final.pth'

In [15]:
cfg['scenes']

'D5,D8,D11'

In [ ]:
cfg['use_template']

In [12]:
cfg

{'data_dir': '/disk1/hogun',
 'seed': 12345,
 'use_template': True,
 'scenes': 'D5,D8,D11',
 'inorder': False,
 'scene_split': 'all',
 'object_split': 'unseen',
 'num_objects': 0,
 'num_scenes': 50,
 'H': 12,
 'W': 15,
 'crop_size': 128,
 'gui_off': True,
 'visualize_graph': False,
 'logging': True,
 'wandb_off': False,
 'algorithm': 'mcts',
 'time_limit': None,
 'iteration_limit': 3000,
 'max_depth': 7,
 'rollout_policy': 'nostep',
 'tree_policy': 'iql',
 'puct_lambda': 0.5,
 'threshold_success': 0.9,
 'threshold_prob': 1e-05,
 'batch_size': 32,
 'binary_reward': False,
 'blurring': 1,
 'exploration': 0.5,
 'gamma': 1,
 'prob_expand': 0.0,
 'normalize_reward': False,
 'reward_type': 'gt',
 'reward_model_path': 'data/classification-best/top_nobg_linspace_mse-best.pth',
 'label_type': 'linspace',
 'view': 'top',
 'qnet_path': '',
 'vnet_path': '',
 'policynet_path': '../policy_learning/logs/0224_1815/pnet_e1.pth',
 'iql_path': '/disk1/hogun/PreferenceDiffusion/iql/logs/0420_1905/iql_fin

In [ ]:
logs = sorted([d for d in os.listdir('data') if d.startswith('mcts') or d.startswith('Sub')])
if after is not None:
    logs = sorted([d for d in logs if d[-9:-5] >= after])
    print('Measure metrics for:', logs)
for logname in logs:
    ep_length = []
    ep_success_length = []
    deltas = []
    scenes = [s for s in os.listdir(os.path.join('data', logname)) if s.startswith('scene')]
    if len(scenes)<5:
        continue
    print(logname)
    for scene in scenes:
        scene_dir = os.path.join('data', logname, scene)
        logfile = [f for f in os.listdir(scene_dir) if f.endswith('.log')]
        if len(logfile)==0:
            continue
        logfile = logfile[0]
        with open(os.path.join(scene_dir, logfile), 'r') as f:
            x = f.readlines()
        times = filter_log_time(x)
        deltas += get_intervals(times)
        scores = filter_log(x)
        if scores is None:
            continue
        if scores[-1]:
            ep_success_length.append(len(scores[:-1]))
        ep_length.append(len(scores[:-1]))
    print('Num episodes:', len(scenes))
    print('Average time:', np.mean(deltas).seconds)
    print('Success rate:', len(ep_success_length)/len(scenes))
    print('Success average length:', np.mean(ep_success_length))
    print('Average length:', np.mean(ep_length))
    print('-'*40)
